<a href="https://colab.research.google.com/github/rsamant07/AdvComputerVision/blob/main/C3_W2_Lab_1_Simple_Object_Detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Simple Object Detection in Tensorflow

This lab will walk you through how to use object detection models available in [Tensorflow Hub](https://www.tensorflow.org/hub). In the following sections, you will:

* explore the Tensorflow Hub for object detection models
* load the models in your workspace
* preprocess an image for inference 
* run inference on the models and inspect the output

Let's get started!

## Imports

In [1]:
import tensorflow as tf
import tensorflow_hub as hub
from PIL import Image
from PIL import ImageOps
import tempfile
from six.moves.urllib.request import urlopen
from six import BytesIO

## Utilities

In [2]:
def download_and_resize_image(url, new_width=256, new_height=256):
    '''
    Fetches an image online, resizes it and saves it locally.
    
    Args:
        url (string) -- link to the image
        new_width (int) -- size in pixels used for resizing the width of the image
        new_height (int) -- size in pixels used for resizing the length of the image
        
    Returns:
        (string) -- path to the saved image
    '''
    
    
    # create a temporary file ending with ".jpg"
    _, filename = tempfile.mkstemp(suffix=".jpg")
    
    # opens the given URL
    response = urlopen(url)
    
    # reads the image fetched from the URL
    image_data = response.read()
    
    # puts the image data in memory buffer
    image_data = BytesIO(image_data)
    
    # opens the image
    pil_image = Image.open(image_data)
    
    # resizes the image. will crop if aspect ratio is different.
    pil_image = ImageOps.fit(pil_image, (new_width, new_height), Image.ANTIALIAS)
    
    # converts to the RGB colorspace
    pil_image_rgb = pil_image.convert("RGB")
    
    # saves the image to the temporary file created earlier
    pil_image_rgb.save(filename, format="JPEG", quality=90)
    
    print("Image downloaded to %s." % filename)
    
    return filename

In [3]:
def run_detector(detector, path):
    '''
    Runs inference on a local file using an object detection model.
    
    Args:
        detector (model) -- an object detection model loaded from TF Hub
        path (string) -- path to an image saved locally
    '''
    
    # load an image tensor from a local file path
    img = load_img(path)

    # add a batch dimension in front of the tensor
    converted_img  = tf.image.convert_image_dtype(img, tf.float32)[tf.newaxis, ...]
    
    # run inference using the model
    result = detector(converted_img)

    # save the results in a dictionary
    result = {key:value.numpy() for key,value in result.items()}

    # print results
    print("Found %d objects." % len(result["detection_scores"]))

    print(result["detection_scores"])
    print(result["detection_class_entities"])
    print(result["detection_boxes"])


In [4]:
def load_img(path):
    '''
    Loads a JPEG image and converts it to a tensor.
    
    Args:
        path (string) -- path to a locally saved JPEG image
    
    Returns:
        (tensor) -- an image tensor
    '''
    
    # read the file
    img = tf.io.read_file(path)
    
    # convert to a tensor
    img = tf.image.decode_jpeg(img, channels=3)
    
    return img

## Download the model from Tensorflow Hub

Tensorflow Hub is a repository of trained machine learning models which you can reuse in your own projects. You can see the domains covered [here](https://tfhub.dev/) and its subcategories. For this lab, you will want to look at the [image object detection subcategory](https://tfhub.dev/s?module-type=image-object-detection). You can select a model to see more information about it and copy the URL so you can download it to your workspace. We selected a couple of these models for this particular lab and loaded the first one.

In [5]:
# you can switch the commented lines here to pick the other model
module_handle = "https://tfhub.dev/google/faster_rcnn/openimages_v4/inception_resnet_v2/1"
#module_handle = "https://tfhub.dev/google/openimages_v4/ssd/mobilenet_v2/1"

# load the chosen model
detector = hub.load(module_handle).signatures['default']

INFO:tensorflow:Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


Some models in the Tensorflow hub can be used for different tasks. For that, its documentation should show what *signature* to use when running the model. For object detection models, it will by default accept a batch of image tensors and output a dictionary describing the objects detected. If you want to see if a model has more than one signature then you can do something like `print(hub.load(module_handle).signatures.keys())`. In our case, the models we will be using only have the `default` signature so you don't have to worry about other types.

## Download and preprocess an image

We can now get a sample image online and save it locally. We will just use the original width and height of the image but feel free to modify it and see what results you get.

In [7]:
image_url = "https://upload.wikimedia.org/wikipedia/commons/f/fb/20130807_dublin014.JPG"

# download the image and use the original height and width
downloaded_image_path = download_and_resize_image(image_url, 3872, 2592)

Image downloaded to /tmp/tmpp3wnfhe_.jpg.


## Run inference on the image

You can run your detector by calling our utility function. This will print the number of objects found followed by three lists: 

* the detection scores of each object found (i.e. how confident the model is), 
* the classes of each object found, 
* the bounding boxes of each object

You will get to see how to overlay these information on the original image in the next sections and in the assignment.

In [8]:
# runs the object detection model and prints information about the objects found
run_detector(detector, downloaded_image_path)

Found 100 objects.
[0.6532174  0.61050487 0.60152495 0.59255666 0.5917778  0.58154964
 0.5505326  0.49575576 0.47424877 0.47322133 0.44066477 0.4051141
 0.39803803 0.3940655  0.3714858  0.36155996 0.3615076  0.34689227
 0.33362356 0.31252927 0.2887799  0.25758353 0.25748897 0.25196072
 0.24782148 0.23412406 0.20432054 0.20324662 0.17988308 0.17964761
 0.1737451  0.16431497 0.16031103 0.15895295 0.15620388 0.15468779
 0.14754511 0.13622598 0.12740111 0.12555654 0.12102704 0.11813035
 0.11387308 0.11229108 0.11129158 0.09718709 0.09137249 0.08976003
 0.0888024  0.08633561 0.08337442 0.08095147 0.07988831 0.0774143
 0.0773216  0.07631025 0.07507852 0.07386057 0.07233316 0.07204045
 0.07110182 0.06935571 0.06825587 0.06427855 0.0624855  0.06226387
 0.06211156 0.05940022 0.05798778 0.05784462 0.05725535 0.05346677
 0.05304343 0.05242869 0.04892596 0.04813037 0.04578473 0.04424176
 0.04337559 0.04277798 0.04262636 0.04163509 0.04081756 0.03975982
 0.03945961 0.03944291 0.03863973 0.03769058 